# TP Programmation avec Keras - Cas MNIST, niveau de confiance

Dans ce TP, nous allons associer un niveau de confiance à nos prédictions en utilisant la méthode du MC-Dropout. Cette méthode consiste à garder le Dropout actif en phase de test, et d'utiliser la propriété aléatoire du Dropout pour obtenir une variabilité sur les sorties du réseau : une grande variabilité implique un faible niveau de confiance et inversement.

Dans ce TP, des cellules seront laissées à trous, il faudra les compléter suivant les consignes. Elles seront identifiées par le mot **Exercice**. Les **Vérifications** seront effectuées principalement par vous-mêmes, sur la bonne convergence des algorithmes ou leur bon fonctionnement.

Ci-dessous, on importe les bibliothèques qui seront utiles.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tqdm import tqdm

## Mise en place des données

Le code ci-dessous charge les données MNIST.

In [ ]:
#NE PAS MODIFIER

(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()

**Exercice** : Normalisez les données d'entrées en les divisant par 255 et passez les données de sortie sous forme catégorielle (one hot encoding, en utilisant keras.utils.to_categorical)

In [ ]:
#A COMPLETER

**Exercice** : Adaptez le nombre de dimensions de X_train et X_test pour utiliser des couches de convolutions 2D.

In [ ]:
#A COMPLETER

## Modèle Keras

### Création du modèle avec convolution

**Exercice** : Créez un modèle avec Keras que vous appellerez "my_model".

**Instructions spécifiques** : 
- Utilisez le format ci-dessous : ce n'est plus un format séquentiel car il faut une option spécifique pour conserver le Dropout actif lors de la phase de test. Il faut utiliser le format Functional API.
- Inspirez vous du format pour créer en séquence :
  - x = your_layer_1(arguments)(x)
  - x = your_layer_2(arguments)(x)
  - ....
  - outputs = your_final_layer(arguments)(x)
- Pour les couches de Dropout, ajoutez à côté de l'argument x, le mot-clé training = True, qui permet de garder la Dropout actif en phase de prédiction. N'utilisez pas de couche de BatchNormalization avant ou après le Dropout.

In [ ]:
inputs = keras.layers.Input((28,28,1))

x = keras.layers.Conv2D(#A COMPLETER)(inputs)
x = keras.layers.BatchNormalization()(x)
    
#A COMPLETER

outputs = #A COMPLETER

my_model = keras.models.Model(inputs,outputs)


**Exercice** : Affichez la structure de votre modèle avec my_model.summary()

In [ ]:
#A COMPLETER

**Vérification** : Pour l'instant, il suffit qu'il n'y ait pas d'erreur.

### Compilation du modèle

**Exercice** : Compilez le modèle avec l'optimizer que vous souhaitez. Mettez une loss function adaptée ainsi qu'une métrique adaptée.

In [ ]:
#A COMPLETER

**Vérification** : De nouveau, s'il n'y a pas d'erreur et que vous avez suivi les instructions, tout devrait bien se passer.

### Mise en place de l'early stopping

**Exercice** : Définissez un early-stopping.

In [ ]:
#A COMPLETER

## L'apprentissage

**Exercice** : Effectuez l'apprentissage classiquement.

In [ ]:
learning = #A COMPLETER

**Vérification** : La loss function devrait diminuer et l'accuracy augmenter. De même pour le jeu de validation.

**Exercice** : Tracez l'évolution de la fonction de coût et de l'accuracy pour le jeu d'entraînement et pour le jeu de validation.

In [ ]:
#A COMPLETER

## Prédictions avec le modèle

**Exercice** : Prenez un exemple au hasard et affichez plusieurs fois sa prédiction. Vous remarquerez qu'elle est différente à chaque appel de votre réseau de neurones. N'utilisez pas my_model.predict (cela désactive le Dropout pour les versions les plus récentes de tensorflow), appliquez directement my_model à votre exemple : my_model(exemple).

In [ ]:
#A COMPLETER

Nous allons caractériser la variabilité dans les prédictions. Pour ce faire, nous allons utiliser la théorie de l'information pour construire des métriques adaptées à la caractérisation de ces incertitudes (auto-évaluées par le réseau).

Nous allons déjà pour une seul prédiction, caractériser l'incertitude sur cette prédiction en fonction des probabilités associées à chaque classe. L'idée est que, si la prédiction associe une forte probabilité sur une classe et une probabilité faible sur les autres classes, la prédiction est "sûre d'elle". Inversement, si elle associe une faible probabilité à toutes les classe, cette prédiction est moins certaine.

Cet aspect se caractérise par **l'entropie de Shannon**, définie de la manière suivante : 

\begin{equation}
\mathcal{H}(\hat{Y}) = -\sum_{i = 1}^{K} \hat{y_i}\log(\hat{y_i}) 
\end{equation}

Ici l'indice $i$ parcourt l'ensemble des classes considérées dans le problème, $\hat{y}$ est la prédiction.

**Exercice** : Codez l'entropie de Shannon ci-dessous. Considérez que $y$ est un tableau à plusieurs dimensions et que l'on souhaite calculer l'entropie suivant un axe particulier (argument ax) qui représente les différentes classes.

In [ ]:
def shannon_entr(y,ax):

  entr = #A COMPLETER

  return entr

**Vérification** : Exécutez la cellule suivante.

In [ ]:
#NE PAS MODIFIER

np.random.seed(seed = 1)

y_hat = np.random.rand(3,10)

print(shannon_entr(y_hat,1))

Le résultat doit être [2.84552209 2.71503273 1.79409548]

Maintenant, nous allons effectuer plusieurs prédictions pour un même exemple, nous aurons une variabilité avec le Monte-Carlo Dropout. L'incertitude totale sera caractérisée par l'entropie de Shannon sur la prédiction moyenne. L'incertitude intrinsèque aux données caractérisée par le réseau (incertitude dite aléatoire) sera donnée par la moyenne des entropies de Shannon. Enfin, l'incertitude dûe à la variabilité des modèle sera donnée par la différence entre les deux quantités précédemment calculées.

Mathématiquement, cela donne :

  - $\mathcal{H}(\mathbb{E}_{w}(\hat{Y}))$ est l'incertitude totale (l'indice $w$ signifie que l'espérance est donnée par la variabilité engendrée par les poids du réseau avec le MC-Dropout)
  - $\mathbb{E}_{w}(\mathcal{H}(\hat{Y}))$ est l'incertitude aléatoire
  - $\mathcal{I}(\hat{Y};w) = \mathcal{H}(\mathbb{E}_{w}(\hat{Y})) - \mathbb{E}_{w}(\mathcal{H}(\hat{Y}))$ est l'incertitude épistémique. On appelle cette quantité l'information mutuelle et caractérise le lien entre la prédiction et la variabilité des poids du réseau dûe au MC-Dropout.

**Exercice** : Prenez un exemple, dupliquez le une centaine de fois sur l'axe 0 à l'aide de la fonction np.repeat.

In [ ]:
i = 0

X_test_i = X_test[i:(i+1)]

X_test_dup = #A COMPLETER

**Exercice** : Effectuez une prédiction sur cet exemple dupliqué.

In [ ]:
Y_pred_dup = #A COMPLETER

**Exercice** : À partir de cette prédiction, calculez l'incertitude aléatoire. Pour rappel, avec la formule donnée ci-dessus, l'incertitude alétoire correspond à la moyenne des entropies de Shannon sur les différentes prédictions.

In [ ]:
incert_aleat = #A COMPLETER

print(incert_aleat)

**Exercice** : De même, à partir de la prédiction, calculez l'incertitude totale : elle correspond à l'entropie de Shannon calculée sur la moyenne des différentes prédictions.

In [ ]:
incert_tot = #A COMPLETER

print(incert_tot)

**Exercice** : Enfin, calculez la part épistémique, qui correspond donc à la différence entre l'incertitude totale et la part aléatoire.

In [ ]:
incert_epist = #A COMPLETER

print(incert_epist)

### Essais sur l'ensemble de la base de test

Le code ci-dessous permet de dupliquer l'ensemble de la base de test une centaine de fois. Il duplique dans un premier temps le vecteur de test le long d'un axe supplémentaire, puis exécute un reshape pour avoir un tableau de dimensions (nombre d'exemples x m_c, dimensions des images).

In [ ]:
#NE PAS MODIFIER

n_mc = 100

X_test_tot_dup = np.expand_dims(X_test,axis = 1)

X_test_tot_dup = np.repeat(X_test_tot_dup,n_mc,axis = 1)

X_test_tot_dup = np.reshape(X_test_tot_dup,(n_mc*X_test.shape[0],X_test.shape[1],X_test.shape[2],X_test.shape[3]))

La fonction ci-dessous permettra d'effectuer des prédictions sur des batchs de données en utilisant directement my_model et non my_model.predict. Cela évite de saturer la mémoire de la machine.

In [ ]:
#NE PAS MODIFIER

def predict_on_batch_with_dropout(model, data, batch_size):
    predictions = []
    for i in tqdm(range(0, len(data), batch_size)):
        batch = data[i:i+batch_size]
        batch_predictions = model(batch, training=True)
        predictions.append(batch_predictions)
    return np.concatenate(predictions, axis=0)

**Exercice** : Effectuez une prédiction sur cette base de test dupliquée en utilisant la fonction predict_on_batch_with_dropout.

In [ ]:
Y_pred_tot = #A COMPLETER

La taille de ce tableau de prédiction est maintenant (n_exemples x n_mc, 10). Pour faire un calcul de l'entropie de Shannon, il faut rassembler les prédictions correspondant au même exemple dans une même dimension : l'idée est d'obtenir un tableau de taille finale (n_exemples, n_mc, 10).

**Exercice** : Utilisez la fonction np.reshape pour atteindre cette taille.

In [ ]:
Y_pred_tot = #A COMPLETER

**Exercice** : Calculez l'incertitude aléatoire pour l'ensemble des prédictions. Votre résultat doit être un vecteur de taille n_exemples (de taille 10 000).

**Hint** : La difficulté principale à gérer est l'axe sur lequel il faut calculer l'entropie puis celui sur lequel il faut calculer la moyenne.

In [ ]:
incert_aleat_tot = #A COMPLETER

print(incert_aleat_tot.shape)

**Exercice** : De même, effectuez le calcul de l'incertitude totale pour l'ensemble des prédictions. Stockez aussi le vecteur donnant la moyenne des prédictions dans le vecteur Y_mean.

In [ ]:
Y_mean = #A COMPLETER

incert_totale_tot = #A COMPLETER

print(incert_totale_tot.shape)

**Exercice** : Enfin, calculez la part épistémique.

In [ ]:
incert_epist_tot = #A COMPLETER

**Exercice** : Classez les exemples selon la valeur de leur incertitude aléatoire et visualisez ceux avec la plus grande incertitude aléatoire. La fonction np.argsort sera utile.

In [ ]:
index_sort = #A COMPLETER

index = #A COMPLETER : stockez l'index que vous voulez visualiser dans cette variable

label_pred = np.argmax(Y_mean[index])

figure = plt.figure(figsize = (16,9))

ax1 = plt.subplot(121)
ax1.imshow(X_test[index,:,:],cmap = "hot")
plt.title("Prédiction moyenne : " + str(label_pred) + "\n Vraie valeur : " + str(Y_test[index]))

ax2 = plt.subplot(122)
ax2.bar(np.arange(10),height = Y_mean[index],tick_label = np.arange(10))
plt.xlabel("Valeur")
plt.ylabel("Output du réseau")
plt.title("Incertitude aléatoire : " + str(incert_aleat_tot[index]) + 
          "\nIncertitude épistémique : " + str(incert_epist_tot[index])+
          "\nIncertitude totale : " + str(incert_totale_tot[index]))


**Exercice** : Faites de même pour l'incertitude épistémique

In [ ]:
index_sort = #A COMPLETER

index = #A COMPLETER : stockez l'index que vous voulez visualiser dans cette variable
 
label_pred = np.argmax(Y_mean[index])

figure = plt.figure(figsize = (16,9))

ax1 = plt.subplot(121)
ax1.imshow(X_test[index,:,:],cmap = "hot")
plt.title("Prédiction moyenne : " + str(label_pred) + "\n Vraie valeur : " + str(Y_test[index]))

ax2 = plt.subplot(122)
ax2.bar(np.arange(10),height = Y_mean[index],tick_label = np.arange(10))
plt.xlabel("Valeur")
plt.ylabel("Output du réseau")
plt.title("Incertitude aléatoire : " + str(incert_aleat_tot[index]) + 
          "\nIncertitude épistémique : " + str(incert_epist_tot[index])+
          "\nIncertitude totale : " + str(incert_totale_tot[index]))

**Exercice** : Enfin, faites de même pour l'incertitude totale.

In [ ]:
index_sort = #A COMPLETER

index =  #A COMPLETER : stockez l'index que vous voulez visualiser dans cette variable

label_pred = np.argmax(Y_mean[index])

figure = plt.figure(figsize = (16,9))

ax1 = plt.subplot(121)
ax1.imshow(X_test[index,:,:],cmap = "hot")
plt.title("Prédiction moyenne : " + str(label_pred) + "\n Vraie valeur : " + str(Y_test[index]))

ax2 = plt.subplot(122)
ax2.bar(np.arange(10),height = Y_mean[index],tick_label = np.arange(10))
plt.xlabel("Valeur")
plt.ylabel("Output du réseau")
plt.title("Incertitude aléatoire : " + str(incert_aleat_tot[index]) + 
          "\nIncertitude épistémique : " + str(incert_epist_tot[index])+
          "\nIncertitude totale : " + str(incert_totale_tot[index]))

Vous pouvez si vous le souhaitez continue d'étudier ces données, en produisant l'histogramme des incertitudes, en regardant la corrélation entre les incertitudes renvoyées et les erreurs du réseau...